In [ ]:
from data_gen import *
from models import *

In [ ]:
dg = DataGen(data="data/wolayitta_clean.txt")

n_features = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len


In [ ]:
print(len(dg.words) * .7)
batch_size = 128
n_batches = int(len(dg.words) * .7 / batch_size) 
gen = dg.cnn_gen_data(batch_size=batch_size, n_batches=n_batches)

In [ ]:
train, infenc, infdec = conv_model(n_features, n_features, dg.word_feat_len, 64, 256)
train.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 0)

In [ ]:
# test_n_batches, test_batch_size =  int(len(dg.words) * .7 / batch_size), batch_size  
test_n_batches, test_batch_size = 30, 100 
test_gen = dg.cnn_gen_data(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

In [ ]:
total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        X33 = X3[j].reshape((1, X3.shape[1])) 
        X11 = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))
        target = predict(infenc, infdec, X11, X33, n_steps_out, n_features)
        root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
        sims.append(dg.word_sim(word, targetS))
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target):
            correct += 1
        else:
            print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
        if root.strip() in targetS.strip():
            in_word += 1
#     print(b, root, word, targetS)
    total += test_batch_size
    
word_sim_average = sum(sims)/len(sims)
print('Word Similarity Average: {0:.2f}%'.format(word_sim_average))
print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
print('Word in Accuracy: %.2f%%' % (float(in_word)/float(total)*100.0))